<a href="https://colab.research.google.com/github/V-Vibee/YBI-Internship/blob/main/Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Modules**

In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=17384c0682664328a60fa19472a40a803f63131b09f8c2060580b73320468a73
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [17]:
import pyspark
import pandas as pd
from datetime import date, datetime

**Initiate Spark session**

In [18]:
from pyspark.sql import SparkSession

**Context**

In [19]:
BDS_Session = SparkSession.builder.getOrCreate()

In [20]:
BDS_Session

In [27]:
pandas_df= pd.DataFrame({
    'a':[1,2,3],
    'b':[2., 3., 4.],
    'c':['string1', 'string2', 'string3'],
    'd':[date(2000,1,1), date(2000,2,1), date(2000,3,1)],
    'e':[datetime(2000,1,1,12,0), datetime(2000,1,2,12,0), datetime(2000,1,3,12,0)]
})

In [29]:
pandas_df

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [30]:
type(pandas_df)

pandas.core.frame.DataFrame

In [32]:
df= BDS_Session.createDataFrame(pandas_df)

In [33]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [42]:
type(df)

pyspark.sql.dataframe.DataFrame

In [35]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [37]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [43]:
df.describe().show()

+-------+---+---+-------+
|summary|  a|  b|      c|
+-------+---+---+-------+
|  count|  3|  3|      3|
|   mean|2.0|3.0|   NULL|
| stddev|1.0|1.0|   NULL|
|    min|  1|2.0|string1|
|    max|  3|4.0|string3|
+-------+---+---+-------+



In [45]:
df.select('a', 'b').show()

+---+---+
|  a|  b|
+---+---+
|  1|2.0|
|  2|3.0|
|  3|4.0|
+---+---+



**Converting to Pandas Dataframe**

In [47]:
df.toPandas()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [51]:
df.filter(df.b==3).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
+---+---+-------+----------+-------------------+



In [61]:
df1= BDS_Session.createDataFrame([
   ['red','banana',1,10], ['blue', 'banana',2,20],
    ['red', 'carrot',3,30],['blue','grape',4,40],
    ['red','carrot',5,50], ['black','carrot',6,60],
    ['red','banana',7,70], ['red', 'grape',8,80] ], schema = ['color', 'fruit', 'v1', 'v2'])

In [62]:
df1.show()


+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [66]:
df1.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [86]:
titanic= pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Titanic.csv')

In [93]:
titanic.to_csv('titanic')

In [97]:
titanic= BDS_Session.read.csv('/content/titanic', header=True, inferSchema= True)

In [98]:
titanic.show()

+---+------+--------+--------------------+------+----+-----+-----+--------+--------+-------+--------+----+-----+--------------------+
|_c0|pclass|survived|                name|   sex| age|sibsp|parch|  ticket|    fare|  cabin|embarked|boat| body|           home.dest|
+---+------+--------+--------------------+------+----+-----+-----+--------+--------+-------+--------+----+-----+--------------------+
|  0|     1|       1|Allen, Miss. Elis...|female|29.0|    0|    0|   24160|211.3375|     B5|       S|   2| NULL|        St Louis, MO|
|  1|     1|       1|Allison, Master. ...|  male|0.92|    1|    2|  113781|  151.55|C22 C26|       S|  11| NULL|Montreal, PQ / Ch...|
|  2|     1|       0|Allison, Miss. He...|female| 2.0|    1|    2|  113781|  151.55|C22 C26|       S|NULL| NULL|Montreal, PQ / Ch...|
|  3|     1|       0|Allison, Mr. Huds...|  male|30.0|    1|    2|  113781|  151.55|C22 C26|       S|NULL|135.0|Montreal, PQ / Ch...|
|  4|     1|       0|Allison, Mrs. Hud...|female|25.0|    1|  